<a href="https://colab.research.google.com/github/RogerioMatos75/colab_colmap/blob/main/colab/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [5]:
# Install tinycudann from the local wheel file
!pip install /content/tinycudann-1.7-cp310-cp310-linux_x86_64.whl

ERROR: tinycudann-1.7-cp310-cp310-linux_x86_64.whl is not a supported wheel on this platform.


In [7]:
# @markdown <h1>Install Nerfstudio and Dependencies (~8 min)</h1>

%cd /content/
!pip install --upgrade pip
# Install torch and torchvision - removing version constraints to allow compatible versions to be installed
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda from Nerfstudio requirements
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git#subdirectory=tinycudann

# Installing COLMAP
%cd /content/
!apt-get install colmap

# Install nerfstudio
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

/content
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
/content
  Cloning https://github.com/nerfstudio-project/nerfstudio.git to /tmp/pip-req-build-j7cbvkr0
  Running command git clone --filter=blob:none --quiet https://github.com/nerfstudio-project/nerfstudio.git /tmp/pip-req-build-j7cbvkr0
  Resolved https://github.com/nerfstudio-project/nerfstudio.git to commit 50e0e3c70c775e89333256213363badbf074f29d
ERROR: git+https://github.com/nerfstudio-project/nerfstudio.git#subdirectory=tinycudann does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
/content
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
colmap is already the newest version (3.7-2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
/content
  Cloning https://github.com/nerfstudio-project/nerfstudio.git to /tmp/pip-req-build-s_kj2xzk
  Running command git clone --filter=blob:none --quiet http

In [8]:
# Build and install tinycudann from source
%cd /content/
!git clone --recursive https://github.com/NVlabs/TinyCUDNN.git
%cd TinyCUDNN/bindings/torch/
!pip install .

/content
Cloning into 'TinyCUDNN'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'TinyCUDNN/bindings/torch/'
/content
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [11]:
# @markdown <h1> Downloading and Processing Data</h1>
# @markdown <h3>Pick the preset scene or upload your own images/video</h3>
import glob
import os

from google.colab import files
from IPython.core.display import HTML, display

scene = "📤 upload your images"  # @param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = " ".join(scene.split(" ")[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == "upload your own Record3D data":
    display(HTML("<h3>Zip your Record3D folder, and upload.</h3>"))
    display(
        HTML(
            '<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'
        )
    )
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob("/content/data/nerfstudio/custom_data/*")[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ["upload your images", "upload your own video"]:
    display(HTML("<h3>Select your custom data</h3>"))
    display(HTML("<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>"))
    display(
        HTML(
            "<p/>Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.<p>"
        )
    )
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == "upload your images":
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == "upload your images":
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content/data/nerfstudio/custom_data/raw_images


Saving frame_0001.png to frame_0001.png
Saving frame_0002.png to frame_0002.png
Saving frame_0003.png to frame_0003.png
Saving frame_0004.png to frame_0004.png
Saving frame_0005.png to frame_0005.png
Saving frame_0006.png to frame_0006.png
Saving frame_0007.png to frame_0007.png
Saving frame_0008.png to frame_0008.png
Saving frame_0009.png to frame_0009.png
Saving frame_0010.png to frame_0010.png
Saving frame_0011.png to frame_0011.png
Saving frame_0012.png to frame_0012.png
Saving frame_0013.png to frame_0013.png
Saving frame_0014.png to frame_0014.png
Saving frame_0015.png to frame_0015.png
Saving frame_0016.png to frame_0016.png
Saving frame_0017.png to frame_0017.png
Saving frame_0018.png to frame_0018.png
Saving frame_0019.png to frame_0019.png
Saving frame_0020.png to frame_0020.png
Saving frame_0021.png to frame_0021.png
Saving frame_0022.png to frame_0022.png
Saving frame_0023.png to frame_0023.png
Saving frame_0024.png to frame_0024.png
Saving frame_0025.png to frame_0025.png


In [12]:
# @markdown <h1>Start Training</h1>

%cd /content
!pip install colab-xterm
%load_ext colabxterm
%env TERM=xterm
from IPython.display import clear_output

clear_output(wait=True)
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    print(
        "\033[1m"
        + "Copy and paste the following command into the terminal window that pops up under this cell."
        + "\033[0m"
    )
    print(
        f"ns-train nerfacto --viewer.websocket-port 7007 --viewer.make-share-url True nerfstudio-data --data data/nerfstudio/{scene} --downscale-factor 4"
    )
    print()
    %xterm
else:
    from IPython.core.display import HTML, display

    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML("<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>"))

In [ ]:
# @title # Render Video { vertical-output: true }
# @markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
# @markdown <h5>The rendered video should be at renders/output.mp4!</h5>


base_dir = "/content/outputs/unnamed/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import HTML, display

display(HTML("<h3>Upload the camera path JSON.</h3>"))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render camera-path --load-config $config_filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4

[19:48:48] Skipping 0 files in dataset split train.                                          ]8;id=527413;file:///content/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=243595;file:///content/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#91\91]8;;\
           Skipping 0 files in dataset split test.                                           ]8;id=109270;file:///content/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=464675;file:///content/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#91\91]8;;\
Loading data batch ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Plea

In [4]:
!apt-get update
!apt-get install unrar

!unrar x /content/frames.rar

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,807 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,681 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:14 https:

In [4]:
# 1. Instala a ferramenta para descompactar arquivos .rar (se ainda não tiver instalado)
!apt-get install unrar

# 2. Descompacta o arquivo. Atenção ao nome exato do arquivo!
# O log diz que ele foi salvo como "frames (1).rar".
!unrar x "frames (1).rar"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal

Cannot open frames (1).rar
No such file or directory
No files to extract


In [9]:
# --- ETAPA 1: INSTALAR DEPENDÊNCIAS BÁSICAS E O PYTORCH ---
%cd /content/
!pip install --upgrade pip
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!apt-get install colmap -y

# --- ETAPA 2: COMPILAR O TINYCUDANN DO CÓDIGO-FONTE ---
print("---===[ Fabricando a peça 'tinycudann' do zero... ]===---")
# Clonar o repositório oficial do tiny-cuda-nn
!git clone https://github.com/NVlabs/tiny-cuda-nn --recursive
%cd tiny-cuda-nn
# Compilar e instalar a biblioteca
!cmake . -B build
!cmake --build build --config Rel --target install

# --- ETAPA 3: INSTALAR O NERFSTUDIO ---
print("\n---===[ Instalando o Nerfstudio... ]===---")
%cd /content/
# Instalar a versão mais recente do Nerfstudio direto do GitHub
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

print("\n\n✅ Ambiente de Batalha Configurado com Sucesso!")

/content
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
colmap is already the newest version (3.7-2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
---===[ Fabricando a peça 'tinycudann' do zero... ]===---
Cloning into 'tiny-cuda-nn'...
remote: Enumerating objects: 4406, done.
remote: Counting objects: 100% (1605/1605), done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 4406 (delta 1437), reused 1363 (delta 1352), pack-reused 2801 (from 2)
Receiving objects: 100% (4406/4406), 19.90 MiB | 22.97 MiB/s, done.
Resolving deltas: 100% (2795/2795), done.
Submodule 'dependencies/cmrc' (https://github.com/vector-of-bool/cmrc) registered for path 'dependencies/cmrc'
Submodule 'dependencies/cutlass' (https://github.com/NVIDIA/cutlass) registered for path 'dependencies/cutlass'
Submodule 'dependencies/fmt' (https://github.com/fm